In [18]:
from pulp import *
import numpy as np
# Define the problem
model = LpProblem("SaaSafrasRevenueMaximization", LpMaximize)

# Decision Variables
# New Business Acquisition (people)
nba = LpVariable('NewBusinessAcquisition', lowBound=0, upBound=20, cat='Integer')

# Account Management (people)
am = LpVariable("AccountManagement", lowBound=0, upBound=20, cat='Integer')

# Support (people)
support = LpVariable("Support", lowBound=0, upBound=20, cat='Integer')

# Constants
revPerCustomer = 100
numCustomerPerAM = 25  # Assuming this value from the second function
churnRate = 0.1  # Assuming initial churn rate
naturalCustomers = 25  # Assuming initial natural customer acquisition
NBAacqCust = 5  # Assuming customer acquisition per new business acquisition
totalrevenue = 0
# Objective Function (Maximize total revenue over 24 months)
for month in range(1, 25):
    existingcustomers = 1000 + naturalCustomers * (month - 1)
    churnreduction = 0.15 * (0.7 + support)
    churnrate = churnRate - churnreduction
    remainingcustomers = existingcustomers + NBAacqCust * nba - churnrate * existingcustomers

    amrevenue = (am * numCustomerPerAM) * revPerCustomer * ((1 + 0.2) ** (min(month + 1, 6)))

    baselinerevenue = 100 * (remainingcustomers - am * numCustomerPerAM)

     # Total Revenue for the month
    monthlyrevenue = amrevenue + baselinerevenue

    totalrevenue += monthlyrevenue

model += lpSum(totalrevenue)

# Constraints on the number of employees
model += nba + am + support <= 20

# Solve the LP problem
model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt']))

# Print the solution status
print(totalrevenue)

108623.99999999991*AccountManagement + 12000*NewBusinessAcquisition + 463500.0*Support + 3105450.0


In [13]:
import numpy as np
jobsMatrix = [[]]
jobsMatrix = np.array([[14, 0, 6],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6],
 [13, 0, 7],
 [13, 1, 6]])


In [128]:
jobsMatrix = [[]]
jobsMatrix = np.array([[0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4],
                      [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4],
                      [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4],
                      [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4],
                      [0, 16, 4], [0, 16, 4], [0, 16, 4], [0, 16, 4]])

In [11]:
def cumRevenue(initChurnRate, numMonths, AMcompoundPerc, naturalCustomers, NBAacqCust, revPerCustomer, jobsMatrix, numCustomerPerAM, initNumCustomers, churnDecreasePerCSAT):
    numCustomers = initNumCustomers
    revPerAMCust =  np.zeros(24)
    for i in range(24):
        revPerAMCust[i] = revPerCustomer*((1+AMcompoundPerc)**(min(i+1,6)))
    numAM = jobsMatrix[:,0]
    numNBA = jobsMatrix[:,1]
    numSupport = jobsMatrix[:,2]
    cumRevenue = 0
    # the first month is the amount of account managers multiplied by the amount of people that each account manager can have under their care
    for month in range(numMonths):
        numCustomers = (numCustomers - numCustomers*max(0, churnRate - (numSupport[month]*churnDecreasePerCSAT*initChurnRate))) + naturalCustomers + NBAacqCust*numNBA[month]
        cumRevenue += ((numCustomers - numCustomerPerAM*numAM[month])*revPerCustomer) + numAM[month]*numCustomerPerAM*revPerAMCust[month]
    return cumRevenue

In [14]:
cumRevenue(initChurnRate=0.10, numMonths = 24, AMcompoundPerc = 0.2, naturalCustomers = 25, NBAacqCust = 5, revPerCustomer=100, jobsMatrix=jobsMatrix, numCustomerPerAM=25, initNumCustomers=1000, churnDecreasePerCSAT=0.15)

4384246.897972111